In [231]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [232]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import string
import glob
from tqdm import tqdm
import random
import pickle
import string

In [233]:
# Get a list of all CSV files in the "Hist_Data" folder
file_list = glob.glob('test folder/*.csv')

# Get the total number of files
total_files = len(file_list)

with open('Pseudonym_Table/timeseries_mapping.pkl', 'rb') as f:
    timeseries_mapping = pickle.load(f)

In [234]:
# Define the time range
train_1_start = '2013-01-01 00:15:00'
train_1_end = '2017-07-01 00:00:00'
test_1_start = '2017-07-01 00:15:00'
test_1_end = '2018-07-01 00:00:00'
train_2_start = '2018-07-01 00:15:00'
train_2_end = '2022-07-01 00:00:00'
c_start = '2022-07-01 00:15:00'
test_2_end = '2023-05-30 23:45:00'

# Create an empty DataFrame with a date range
train_1_date_range = pd.date_range(start=train_1_start, end=train_1_end, freq='15T')
train_1_empty_df = pd.DataFrame(index=train_1_date_range)
train_1_empty_df.index.name = 'Zeitstempel'
train_1_empty_df = train_1_empty_df.reset_index()

test_1_date_range = pd.date_range(start=test_1_start, end=test_1_end, freq='15T')
test_1_empty_df = pd.DataFrame(index=test_1_date_range)
test_1_empty_df.index.name = 'Zeitstempel'
test_1_empty_df = test_1_empty_df.reset_index()

train_2_date_range = pd.date_range(start=train_2_start, end=train_2_end, freq='15T')
train_2_empty_df = pd.DataFrame(index=train_2_date_range)
train_2_empty_df.index.name = 'Zeitstempel'
train_2_empty_df = train_2_empty_df.reset_index()

test_2_date_range = pd.date_range(start=test_2_start, end=test_2_end, freq='15T')
test_2_empty_df = pd.DataFrame(index=test_2_date_range)
test_2_empty_df.index.name = 'Zeitstempel'
test_2_empty_df = test_2_empty_df.reset_index()

In [235]:
def generate_statistics(dataframe):
    if dataframe is df:
        dataset = 'Dataset' + '_' +  str(processed_files)
        filename = 'Dataset_' + alphabetic_id + '_' +  str(processed_files)
    elif dataframe is train_1_original:
        dataset = 'train_1' + '_' +  str(processed_files)
        filename = 'train_1_' + alphabetic_id + '_' +  str(processed_files)
    elif dataframe is test_1_original:
        dataset = 'test_1' + '_' +  str(processed_files)
        filename = 'test_1_' + alphabetic_id + '_' +  str(processed_files)
    elif dataframe is train_2_original:
        dataset = 'train_2' + '_' +  str(processed_files)
        filename = 'train_2_' + alphabetic_id + '_' +  str(processed_files)
    elif dataframe is test_2_original:
        dataset = 'test_2' + '_' +  str(processed_files)
        filename = 'test_2_' + alphabetic_id + '_' +  str(processed_files)
    
    # Number of rows
    num_rows = len(dataframe)

    # Number of non-NA values
    num_non_na = dataframe['Wert'].count()

    # Number of 0 values
    num_zeros = np.sum(dataframe['Wert'] == 0)

    if dataframe is df:
        # Filter rows where 'Originalstatus' is not empty and not 'W'
        filtered_data = df_original[df_original['Originalstatus'].notna()]
    else:
        filtered_data = dataframe[dataframe['Originalstatus'].notna()]

    # Count the occurrences of each unique letter in 'Originalstatus'
    letter_counts = filtered_data['Originalstatus'].value_counts()

    # Descriptive statistics
    mean_value = dataframe['Wert'].mean()
    std_dev = dataframe['Wert'].std()
    variance = dataframe['Wert'].var()
    percentiles = dataframe['Wert'].quantile([0.05, 0.25, 0.5, 0.75, 0.95])
    min_value = dataframe['Wert'].min()
    max_value = dataframe['Wert'].max()

    # Create a dictionary with the statistics
    statistics = {
        "Dataset": [dataset],  # Use the provided name argument
        "Filename": [filename],
        "ID": [processed_files],
        "Pseudonym": [alphabetic_id],
        "Timeseriesname": [timeseries_name],
        "Number of rows": [num_rows],
        "Number of non-NA values": [num_non_na],
        "Number of 0 values": [num_zeros],
        "Mean": [mean_value],
        "Standard Deviation": [std_dev],
        "Variance": [variance],
        "Min Value": [min_value],
        "Max Value": [max_value]
    }

    # Add error counts by type for 'Dataset' only
    for error_type, count in letter_counts.items():
        column_name = f"Error-Type: {error_type}"
        statistics[column_name] = [count]

    # Create a DataFrame
    stats_df = pd.DataFrame(statistics)

    return stats_df

In [236]:
# Initialize a counter for processed files
processed_files = 0

# Define the base file path
base_path = "Datasets/"

# Iterate through each file
for id, file in tqdm(enumerate(file_list), total=total_files, desc="Processing files"):
    try:
        # Read the CSV file
        df = pd.read_csv(file, delimiter=';', decimal=',')

        # Convert 'Zeitstempel' column to datetime
        df['Zeitstempel'] = pd.to_datetime(df['Zeitstempel'], format='%d.%m.%Y %H:%M')
        
        # Check if the time intervals are 15 minutes apart
        time_diff = df['Zeitstempel'].diff()

        # Print the time differences
        if (time_diff > pd.Timedelta("1 days")).any():
            raise ValueError(f"File {file} doesn't have 15-minute steps.")

        # Get timeseries_name
        timeseries_name = df['Messstelle'].iloc[0]
        for letter, timeseries in timeseries_mapping.items():
            if timeseries_name == timeseries:
                alphabetic_id = letter
                print(f"id = {processed_files}: {alphabetic_id}")
                break

        else:
            raise ValueError(f"No matching timeseries found for {timeseries_name}")

        # Assign df to df_original
        df_original = df.copy()  # Create a copy to avoid modifying df
        df_original

    except ValueError as e:
        print(f"Error processing file {file}: {e}")
        continue 

#------------------------------------------------------------------------------------------------------------------------------    
## Step 1: Insert original values into timeseries and mark the right status 

    # Find rows where "Originalwert" is not NaN and drop the columns afterwards
    if 'Originalwert' in df.columns and 'Originalstatus' in df.columns:
        mask = ~df['Originalwert'].isna() #gives True or False

        # Replace "Wert" with "Originalwert" where "Originalwert" is not NaN (loc gives u the complete row)
        df.loc[mask, 'Wert'] = df.loc[mask, 'Originalwert'] 

        # Set "Status" at index 0 to 0 and all "W"s in "Status" to 1
        df.loc[df['Status'] == 'W', 'Status'] = 1
        df.loc[df['Status'] == 'E', 'Status'] = 0

        df = df.drop(['Originalwert', 'Originalstatus'], axis=1)
    else:
        print("Columns 'Originalwert' and 'Originalstatus' do not exist in the DataFrame.")

#------------------------------------------------------------------------------------------------------------------------------    
## Step 2: Split Dataset into training and test set

    # Define the conditions for each dataset
    train_1_condition = (df['Zeitstempel'] >= train_1_start) & (df['Zeitstempel'] <= train_1_end)
    test_1_condition = (df['Zeitstempel'] >= test_1_start) & (df['Zeitstempel'] <= test_1_end)
    train_2_condition = (df['Zeitstempel'] >= train_2_start) & (df['Zeitstempel'] <= train_2_end)
    test_2_condition = (df['Zeitstempel'] >= test_2_start) & (df['Zeitstempel'] <= test_2_end)

    # Create the datasets
    train_1 = df[train_1_condition]
    test_1 = df[test_1_condition]
    train_2 = df[train_2_condition]
    test_2 = df[test_2_condition]

    # Reset the index for the new datasets
    train_1 = train_1.reset_index(drop=True)
    test_1 = test_1.reset_index(drop=True)
    train_2 = train_2.reset_index(drop=True)
    test_2 = test_2.reset_index(drop=True)
    
    train_1_original =  df_original[train_1_condition]
    test_1_original = df_original[test_1_condition]
    train_2_original = df_original[train_2_condition]
    test_2_original = df_original[test_2_condition]
    

#------------------------------------------------------------------------------------------------------------------------------    
## Step 3: Standardize training set and see if the first trainingset is available

    if train_1.empty is False: 
        #Get Statistics
        mean_wert_train_1 = train_1_original['Wert'].mean()
        mean_wert_train_2 = train_2_original['Wert'].mean()

        #Scale the data
        train_1['Wert'] = train_1['Wert'].apply(lambda x: 0 if x == 0 else round(x / mean_wert_train_1, 7))
        train_2['Wert'] = train_2['Wert'].apply(lambda x: 0 if x == 0 else round(x / mean_wert_train_2, 7))

        ## Step 4: Apply Standardization to test set

        # Scale the data
        test_1['Wert'] = test_1['Wert'].apply(lambda x: 0 if x == 0 else round(x / mean_wert_train_1, 7))
        test_2['Wert'] = test_2['Wert'].apply(lambda x: 0 if x == 0 else round(x / mean_wert_train_2, 7))
        
        ## Step 5: Store train and testset and make a list for remaining datasets
        
        # Copy relevant columns
        train_1_subset = train_1[['Zeitstempel','Wert', 'Status']].copy()
        test_1_subset = test_1[['Zeitstempel','Wert', 'Status']].copy()
        train_2_subset = train_2[['Zeitstempel','Wert', 'Status']].copy()
        test_2_subset = test_2[['Zeitstempel','Wert', 'Status']].copy()
        
        #Merge empty df with the subsets to get the whole dataset
        train_1_merged = pd.merge(train_1_empty_df, train_1_subset, on='Zeitstempel', how='outer')
        train_1_merged = train_1_merged.drop(['Zeitstempel'], axis=1)
        
        test_1_merged = pd.merge(test_1_empty_df, test_1_subset, on='Zeitstempel', how='outer')
        test_1_merged = test_1_merged.drop(['Zeitstempel'], axis=1)
        
        train_2_merged = pd.merge(train_2_empty_df, train_2_subset, on='Zeitstempel', how='outer')
        train_2_merged = train_2_merged.drop(['Zeitstempel'], axis=1)
        
        test_2_merged = pd.merge(test_2_empty_df, test_2_subset, on='Zeitstempel', how='outer')
        test_2_merged = test_2_merged.drop(['Zeitstempel'], axis=1)

        datasets_original = [df, train_1_original, test_1_original, train_2_original, test_2_original]
        datasets = [ train_1_merged, test_1_merged, train_2_merged, test_2_merged]

        # Define the subfolders and corresponding DataFrame names
        subfolders = ['train_1', 'test_1', 'train_2', 'test_2']

        # Combine subfolders with corresponding DataFrames
        folders_data = zip(subfolders, datasets)

        # Iterate over the combined data
        for folder, data in folders_data:
            # Define the file path
            file_path = f"{base_path}{folder}/{folder}_{alphabetic_id}_{processed_files}.csv"

            # Export the DataFrame to a CSV file
            data.to_csv(file_path, sep=';', decimal=',', index=False)
        
    else:
        test_1.drop(test_1.index, inplace=True)
        mean_wert_train_2 = train_2_original['Wert'].mean()
        train_2['Wert'] = train_2['Wert'].apply(lambda x: 0 if x == 0 else round(x / mean_wert_train_2, 7))
        
        ## Step 4: Apply Standardization to test set

        # Scale the data
        test_2['Wert'] = test_2['Wert'].apply(lambda x: 0 if x == 0 else round(x / mean_wert_train_2, 7))
        
        ## Step 5: Store train and testset and make a list for remaining datasets
        
        # Copy relevant columns
        train_2_subset = train_2[['Zeitstempel','Wert', 'Status']].copy()
        test_2_subset = test_2[['Zeitstempel','Wert', 'Status']].copy()
        
        #Merge empty df with the subsets to get the whole dataset    
        train_2_merged = pd.merge(train_2_empty_df, train_2_subset, on='Zeitstempel', how='outer')
        train_2_merged = train_2_merged.drop(['Zeitstempel'], axis=1)
        
        test_2_merged = pd.merge(test_2_empty_df, test_2_subset, on='Zeitstempel', how='outer')
        test_2_merged = test_2_merged.drop(['Zeitstempel'], axis=1)
        
        datasets_original = [df, train_2_original, test_2_original]
        datasets = [train_2_merged, test_2_merged]

        # Define the subfolders and corresponding DataFrame names
        subfolders = ['train_2', 'test_2']

        # Combine subfolders with corresponding DataFrames
        folders_data = zip(subfolders, datasets)

        # Iterate over the combined data
        for folder, data in folders_data:
            # Define the file path
            file_path = f"{base_path}{folder}/{folder}_{alphabetic_id}_{processed_files}.csv"

            # Export the DataFrame to a CSV file
            data.to_csv(file_path, sep=';', decimal=',', index=False)

#------------------------------------------------------------------------------------------------------------------------------    
## Step 7: Get Statistics from the whole dataset 
    final_stats_df = pd.DataFrame()  # Initialize an empty DataFrame

    for dataset in datasets_original:
        stats_df = generate_statistics(dataset)
        final_stats_df = pd.concat([final_stats_df, stats_df])

    # Reset the index of the final DataFrame
    final_stats_df.reset_index(drop=True, inplace=True)

    # Save the resulting DataFrame
    file_path_stats = f"Statistics/statistics_{alphabetic_id}_{processed_files}.csv"
    
    # Export the DataFrame to a CSV file
    final_stats_df.to_csv(file_path_stats, sep=';', decimal=',', index = False)

    # Increment the counter for processed files
    processed_files += 1
    
final_stats_df


Processing files:   0%|          | 0/7 [00:00<?, ?it/s]

id = 0: BQ


Processing files:  14%|█▍        | 1/7 [00:03<00:23,  3.96s/it]

id = 1: BT


Processing files:  29%|██▊       | 2/7 [00:07<00:19,  3.84s/it]

...

id = 4: FBM


Processing files:  86%|████████▌ | 6/7 [00:12<00:01,  1.75s/it]

id = 5: FBM


Processing files: 100%|██████████| 7/7 [00:14<00:00,  2.08s/it]


,Dataset,Filename,ID,Pseudonym,Timeseriesname,Number of rows,Number of non-NA values,Number of 0 values,Mean,Standard Deviation,Variance,Min Value,Max Value,Error-Type: W,Error-Type: F
0,Dataset_5,Dataset_FBM_5,5,FBM,UER4601903,195647,195647,11047,46.345727,12.318944,151.756382,0.0,235.00,3140.0,16.0
1,train_1_5,train_1_FBM_5,5,FBM,UER4601903,157632,157632,7560,46.966515,11.733341,137.671302,0.0,114.00,3140.0,16.0
2,test_1_5,test_1_FBM_5,5,FBM,UER4601903,35040,35040,317,47.681854,5.388750,29.038622,0.0,102.00,NaN,NaN
3,train_2_5,train_2_FBM_5,5,FBM,UER4601903,2975,2975,27,48.795745,5.370474,28.841988,0.0,66.05,NaN,NaN
4,test_2_5,test_2_FBM_5,5,FBM,UER4601903,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
